<a href="https://colab.research.google.com/github/siqisiqisiqisiqi/deep-learning-hw1/blob/main/hw1_2_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [158]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Normalize 
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.modules.conv import Conv2d
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [2]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10
random_seed = 1
torch.manual_seed(random_seed)

In [3]:
training_data = datasets.MNIST(
    root = "data",
    train = True,
    download = True,
    transform=Compose([ToTensor(),Normalize((0.1307,),(0.3081,))])
)
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=Compose([ToTensor(),Normalize((0.1307,),(0.3081,))])
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [4]:
train_loader = DataLoader(training_data, batch_size=batch_size_train, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size_test, shuffle=True)

In [5]:
class DNN_NeuralNetwork(nn.Module):
  def __init__(self):
    super(DNN_NeuralNetwork, self).__init__()
    self.l1 = torch.nn.Linear(784, 128)
    self.l2 = torch.nn.Linear(128, 64)
    self.l3 = torch.nn.Linear(64, 16)
  def forward(self, x):
    x = x.view(-1, 784)
    x = F.relu(self.l1(x))
    x = F.relu(self.l2(x))
    x = F.relu(self.l3(x))
    return F.log_softmax(x)

In [6]:
def calculate_gadient_norm():
  grad_all = 0.0
  for p in network.parameters():
    grad = 0.0
    if p.grad is not None:
      grad = np.array((p.grad.cpu().data.numpy()**2).sum())
    grad_all = grad_all + grad
  return grad_all ** 0.5

In [7]:
class gradientnormLossfunction(nn.Module):
  def __init__(self):
    super(gradientnormLossfunction, self).__init__()
    return

  def forward(self, model):
    grad_all = np.array(0.0)
    grad_all = torch.from_numpy(grad_all)
    grad_all.requires_grad_(True)
    for p in model.parameters():
      grad = 0.0
      if p.grad is not None:
        grad = (p.grad.cpu().data.numpy()**2).sum()
      grad_all = grad_all + grad

    return grad_all ** 0.5

In [48]:
def Accuracy(dataloader):
  network.eval()
  correct = 0
  with torch.no_grad():
    for data, target in dataloader:
      output = network(data)
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  accu = 100. * correct / len(dataloader.dataset)
  return accu

In [9]:
loss_gradient_fn = gradientnormLossfunction()

In [173]:
def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    
    output = network(data)
    loss = F.nll_loss(output, target)
    # loss = loss_gradient_fn(network)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      train_losses.append(loss.item())
      gradient = calculate_gadient_norm()
      grad_norm.append(gradient)
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      percent = np.around(100. * batch_idx / len(train_loader))

      if percent % 20 == 0.0:
        weight = network.l1.weight
        weight = weight.detach().numpy()
        flatten_weight = weight.reshape(1,-1)
        accu = Accuracy(train_loader)
        PCA_data.append(flatten_weight)
        accu_data.append(accu)
      # torch.save(network.state_dict(), './model.pth')
      # torch.save(optimizer.state_dict(), './optimizer.pth')

In [174]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

In [175]:
train_losses = []
test_losses = []
grad_norm = []
PCA_data = []
accu_data = []
network = DNN_NeuralNetwork()
loss_fn = torch.nn.CrossEntropyLoss()
loss_gradient_fn = gradientnormLossfunction()

n_epochs = 1

optimizer = torch.optim.SGD(network.parameters(), lr= 0.01, momentum= momentum)
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.824273
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.775206
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.748669
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.696835
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.628349
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.568216
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.430449
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.235202
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.086451
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.802999
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.582335
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.251001
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.151170
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.986681
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.125499
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.002882
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.735278
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.681258
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.815764
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.586692
Train Epoch: 1 [12800/60000 (

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.2859, Accuracy: 9162/10000 (92%)



In [166]:
PCA_data1 = np.array(PCA_data)
PCA_data1 = np.squeeze(PCA_data)
# PCA_data1 = sc.fit_transform(PCA_data1)
# pca = PCA(n_components=2)
# dst_data = pca.fit_transform(PCA_data1)
print(PCA_data1.shape)

(4, 1024)


In [ ]:
plt.subplot(2,1,1)
plt.plot(grad_norm)
plt.xlabel("iteration")
plt.ylabel("grad norm")

plt.subplot(2,1,2)
plt.plot(train_losses)
plt.xlabel("iteration")
plt.ylabel("loss")